# Laboratorium 3 - MOwNiT2

## Układy równań liniowych

In [2]:
from typing import Optional, Tuple
import numpy as np

### Zadanie rozgrzewkowe
Napisać mnożenie macierzy w ulubionym_\**_ języku programowania.

In [4]:
def agh_superfast_matrix_multiply(a: np.matrix, b: np.matrix) -> np.matrix:
    """Perform totally ordinary multiplication of matrices.
    
    :param a: matrix with dimensions n by m
    :param b: matrix with dimensions m by p
    :return:  matrix with dimensions n by p
    """
    n = a.shape[0]
    m = a.shape[1]
    p = b.shape[1]
    c = np.zeros((n,p))
    
    for i in range(0,n):
        for j in range (0,p):
            for k in range(0,m):
                c[i,j] += a[i,k] * b[k,j]
    return c
    pass

m1 = np.matrix([[1, 2],
                [3, 4],
                [4, 5],
                [5, 1]])

m2 = np.matrix([[1, 2, 3],
                [4, 5, 6]])

res = agh_superfast_matrix_multiply(m1, m2)
assert np.allclose(res, m1 * m2), "Wrong multiplication result"

### Zadanie 1
1. Napisać kod (w ulubionym_\**_ języku) do eliminacji Gaussa z i bez pivotingu.
2. Rozwiązać poniższy układ równań z pivotingiem i bez, porównać wyniki.

#### Gauss bez pivotingu

In [13]:
def naive_gauss(a, b):
    n = a.shape[0]
    x = np.zeros(n)
    
    for k in range(0,n):
        for i in range (k+1, n):
            xmult = a[i,k] / a[k,k]
            a[i,k] = xmult
            for j in range(k+1, n):
                a[i,j] -= xmult * a[k,j]
            b[i] -= xmult * b[k]
    
    x[n-1] = b[n-1] / a[n-1,n-1]
    for i in range(n-1,-1,-1):
        sum = b[i]
        for j in range(i+1,n):
            sum -= a[i,j] * x[j]
        x[i] = sum / a[i,i]
        
    return np.matrix(x).transpose()
    pass

In [16]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])
b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()

x = np.linalg.solve(A, b)
x_ng = naive_gauss(A, b)

np.allclose(x_ng, x)

True

#### Gauss z pivotingiem

In [36]:
def pivot_gauss(a, b):
    n = a.shape[0]
    x = np.zeros(n)
    s = np.zeros(n)               #tablica z największymi wartościami w danym rzędzie
    r = np.zeros(n, dtype=np.int) #wektor wskazujący rzędy
    
    #znalezienie maksymalnego elementu w każdym rzędzie
    for i in range(0,n):
        r[i] = i;
        smax = 0;
        for j in range(0,n):
            smax = max(smax, abs(a[i,j]))
        s[i] = smax
    
    #ustawienie rzędów w kolejności od największego do najmniejszego
    for k in range(0,n-1):
        rmax = 0
        for i in range(k,n):
            p = abs(a[r[i],k] / s[r[i]])
            if p > rmax:
                rmax = p
                j = i
        tmp = r[j]
        r[j] = r[k]
        r[k] = tmp
    
    #zwykły gauss tylko z wykorzystaniem wektora na odpowiednie rzędy
    for k in range(0,n-1):   
        for i in range(k+1,n):
            xmult = a[r[i],k] / a[r[k],k]
            a[r[i],k] = xmult
            for j in range(k+1,n):
                a[r[i],j] -= xmult * a[r[k],j]
            b[r[i]] -= a[r[i],k] * b[r[k]]
                
    x[r[n-1]] = b[r[n-1]] / a[r[n-1],n-1]
    for i in range(n-1,-1,-1):
        sum = b[r[i]]
        for j in range(i+1,n):
            sum -= a[r[i],j] * x[j]
        x[i] = sum / a[r[i],i]    
    
    return np.matrix(x).transpose()
    pass

In [37]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])
b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()

x = np.linalg.solve(A, b)
x_pg = pivot_gauss(A, b)

np.allclose(x_pg, x)

True

#### Porównanie czasów

In [39]:
%timeit np.linalg.solve(A,b)
%timeit naive_gauss(A,b)
%timeit pivot_gauss(A,b)

13 µs ± 60.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
323 µs ± 6.33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
404 µs ± 5.49 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Zadanie 2
1. Zaimplementować algorytm faktoryzacji LU macierzy.
2. Zaimplementować funkcję sprawdzającą, czy dana macierz jest symetryczna i dodatnio określona.
3. Zaimplementować algorytm faktoryzacji Cholesky'ego macierzy.

In [68]:
def agh_superfast_lu(a: np.matrix) -> Optional[Tuple[np.matrix, np.matrix]]:
    n = a.shape[0]
    l = np.zeros((n,n))
    u = np.zeros((n,n))
    
    for i in range(0,n):
        l[i,i] = 1
        for j in range(i,n):
            sum = 0
            for k in range(0,i):
                sum += l[i,k] * u[k,j]
            u[i,j] = a[i,j] - sum
        for j in range(i+1,n):
            sum = 0
            for k in range(0,i):
                sum += l[j,k] * u[k,i]
            l[j,i] = (a[j,i] - sum) / u[i,i]
    return (l, u)
    pass

def agh_superfast_check_spd(a: np.matrix) -> bool:
    l = agh_superfast_cholesky(a)
    llt = agh_superfast_matrix_multiply(l,l.transpose())
    return np.allclose(a, llt)
    pass

def agh_superfast_cholesky(a: np.matrix) -> Optional[np.matrix]:
    n = a.shape[0]
    l = np.zeros((n,n))
    
    for i in range(0,n):
        sum = 0
        for k in range(0,i):
            sum += l[i,k] * l[i,k]
        l[i,i] = pow(a[i,i] - sum , 1/2)
        for j in range(i+1, n):
            sum = 0
            for k in range(0,i):
                sum += l[j,k] * l[i,k]
            l[j,i] = (a[j,i] - sum) / l[i,i]
    return l
    pass

In [ ]:
Sprawdzenie działania 

In [84]:
A = np.matrix([[2, -1, 0],
               [-1, 2, -1],
               [0, -1, 2]])

LU = agh_superfast_lu(A)
print(LU[0], " = L")
print(LU[1], " = U")

[[ 1.          0.          0.        ]
 [-0.5         1.          0.        ]
 [ 0.         -0.66666667  1.        ]]  = L
[[ 2.         -1.          0.        ]
 [ 0.          1.5        -1.        ]
 [ 0.          0.          1.33333333]]  = U


In [85]:
A = np.matrix([[2, -1, 0],
               [-1, 2, -1],
               [0, -1, 2]])

agh_superfast_check_spd(A)

True

In [86]:
A = np.matrix([[2, -1, 0],
               [-1, 2, -1],
               [0, -1, 2]])

L = agh_superfast_cholesky(A)
print(L, " = L")
print(L.transpose(), " = L^T")

[[ 1.41421356  0.          0.        ]
 [-0.70710678  1.22474487  0.        ]
 [ 0.         -0.81649658  1.15470054]]  = L
[[ 1.41421356 -0.70710678  0.        ]
 [ 0.          1.22474487 -0.81649658]
 [ 0.          0.          1.15470054]]  = L^T
